In [2]:
import numpy as np
import os

In [ ]:
standard_videos = []
standard_configs = {
    'villegas_combined_conv4' : [
        'adam_l2_100_feat',
        'rgb_L_13_cropped_consistent'
    ],
    'villegas_combined' : [
        'rgb_L_13_cropped_consistent'
    ]
}